In [1]:
from baseline_cnn import *

In [2]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [3]:
# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

net=Nnet().to(computing_device)
net.apply(weights_init)

# Print the model
print(net)

#loss criteria are defined in the torch.nn package
criterion = nn.CrossEntropyLoss()


#Instantiate the gradient descent optimizer - use Adam optimizer with default parameters
optimizer = optim.Adam(net.parameters(),lr = 0.001)

CUDA is supported
Nnet(
  (main): Sequential(
    (0): Conv2d(3, 21, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): ReLU(inplace=True)
    (2): Conv2d(21, 20, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU(inplace=True)
    (5): Conv2d(20, 15, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU(inplace=True)
    (8): Conv2d(15, 7, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
  )
  (fc): Sequential(
    (0): Linear(in_features=1183, out_features=500, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=500, out_features=201, bias=True)
  )
)


In [4]:
transform = transforms.Compose([transforms.Resize(224), transforms.CenterCrop(224), transforms.ToTensor()])
dataset = loader('train.csv','/datasets/cs154-fa19-public/',transform=transform)
batch_size = 64
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)

In [ ]:
# Track the loss across training
total_loss = []
avg_minibatch_loss = []
N = 50

for epoch in range(50):
    N_minibatch_loss = 0.0
    correct = 0
    total = 0

    # Get the next minibatch of images, labels for training
    for minibatch_count, (images, labels) in enumerate(train_loader, 0):
        #print("mini_batch", minibatch_count)
        # Zero out the stored gradient (buffer) from the previous iteration
        optimizer.zero_grad()
        # Put the minibatch data in CUDA Tensors and run on the GPU if supported
        images, labels = images.to(computing_device), labels.to(computing_device)
        # Perform the forward pass through the network and compute the loss
        outputs = net(images)
        
        loss = criterion(outputs, labels)
        # Automagically compute the gradients and backpropagate the loss through the network
        loss.backward()

        # Update the weights
        optimizer.step()    
        # Add this iteration's loss to the total_loss
        total_loss.append(loss.item())
        N_minibatch_loss += loss
               
        
        if minibatch_count % N == 49:
            #Print the loss averaged over the last N mini-batches
            N_minibatch_loss /= N
            print('Epoch %d, average minibatch %d loss: %.3f' % (epoch + 1, minibatch_count+1, N_minibatch_loss))
            # Add the averaged loss over N minibatches and reset the counter
            avg_minibatch_loss.append(N_minibatch_loss)
            N_minibatch_loss = 0.0

    print("Finished", epoch + 1, "epochs of training")
    # TODO: Implement validation #with torch.no_grad():
    
    with torch.no_grad():
        for minibatch_count, (images, labels) in enumerate(validation_loader, 0):
            # Put the minibatch data in CUDA Tensors and run on the GPU if supported
            images, labels = images.to(computing_device), labels.to(computing_device)
            for index in range(N):    
                real_y = labels[index]
                #print('real_y', real_y)
                out = net(images[index].view(-1,3,224,224))
                #print('out:', out)
                pred_y = torch.argmax(out)
                #print('pred_y', pred_y)

                if (real_y.item() == pred_y.item()):
                    correct += 1
                total += 1
    print('Accuracy: %.3f' % ((correct/total)*100))

Epoch 1, average minibatch 50 loss: 5.182
Epoch 1, average minibatch 100 loss: 4.753
Epoch 1, average minibatch 150 loss: 4.308
Epoch 1, average minibatch 200 loss: 3.966
Epoch 1, average minibatch 250 loss: 3.705
Epoch 1, average minibatch 300 loss: 3.452
Epoch 1, average minibatch 350 loss: 3.259
Finished 1 epochs of training
Accuracy: 22.021
Epoch 2, average minibatch 50 loss: 2.734
Epoch 2, average minibatch 100 loss: 2.590
Epoch 2, average minibatch 150 loss: 2.569
Epoch 2, average minibatch 200 loss: 2.403
Epoch 2, average minibatch 250 loss: 2.361
Epoch 2, average minibatch 300 loss: 2.279
Epoch 2, average minibatch 350 loss: 2.303
Finished 2 epochs of training
Accuracy: 35.468
Epoch 3, average minibatch 50 loss: 1.799
Epoch 3, average minibatch 100 loss: 1.698
Epoch 3, average minibatch 150 loss: 1.744
Epoch 3, average minibatch 200 loss: 1.693
Epoch 3, average minibatch 250 loss: 1.725
Epoch 3, average minibatch 300 loss: 1.640
Epoch 3, average minibatch 350 loss: 1.644
Finish